In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


./main.ipynb
./.git/config
./.git/HEAD
./.git/description
./.git/index
./.git/packed-refs
./.git/FETCH_HEAD
./.git/objects/pack/pack-f732ccd3bc8575f6e3a3a0c50bb8c34e25793e43.pack
./.git/objects/pack/pack-f732ccd3bc8575f6e3a3a0c50bb8c34e25793e43.idx
./.git/info/exclude
./.git/logs/HEAD
./.git/logs/refs/heads/RecommenderModel
./.git/logs/refs/heads/Dev
./.git/logs/refs/heads/main
./.git/logs/refs/remotes/origin/HEAD
./.git/logs/refs/remotes/origin/RecommenderModel
./.git/logs/refs/remotes/origin/Dev
./.git/hooks/commit-msg.sample
./.git/hooks/pre-rebase.sample
./.git/hooks/pre-commit.sample
./.git/hooks/applypatch-msg.sample
./.git/hooks/fsmonitor-watchman.sample
./.git/hooks/pre-receive.sample
./.git/hooks/prepare-commit-msg.sample
./.git/hooks/post-update.sample
./.git/hooks/pre-merge-commit.sample
./.git/hooks/pre-applypatch.sample
./.git/hooks/pre-push.sample
./.git/hooks/update.sample
./.git/hooks/push-to-checkout.sample
./.git/refs/heads/RecommenderModel
./.git/refs/heads/Dev
./.gi

# Load Data

In [14]:
books = pd.read_csv('/Users/thabochesane/Desktop/path2/books_data.csv')
reviews = pd.read_csv('/Users/thabochesane/Desktop/path2/Books_rating.csv')





In [15]:
books.shape, reviews.shape

((212404, 10), (3000000, 10))

# Filtering

In [16]:
reviews = reviews[['Id','User_id','review/score','Title']]

In [17]:
reviews.rename(columns={'Id' : 'BookId', 'User_id' : 'UserId', 'review/score' : 'Review' , 'Title' : 'BookTitle' }, inplace= True)

In [18]:
reviews.head(100)

,BookId,UserId,Review,BookTitle
0,1882931173,AVCGYZL8FQQTD,4.0,Its Only Art If Its Well Hung!
1,0826414346,A30TK6U7DNS82R,5.0,Dr. Seuss: American Icon
2,0826414346,A3UH4UZ4RSVO82,5.0,Dr. Seuss: American Icon
3,0826414346,A2MVUWT453QH61,4.0,Dr. Seuss: American Icon
4,0826414346,A22X4XUPKF66MR,4.0,Dr. Seuss: American Icon
...,...,...,...,...
95,B000NKGYMK,A2UMP9TJTJ6A6B,1.0,Alaska Sourdough
96,B000NKGYMK,AC2TK7NHKB5C0,5.0,Alaska Sourdough
97,B000NKGYMK,A22T74YNRM8NTK,5.0,Alaska Sourdough
98,B000NKGYMK,A2E0GB5QZRX2JZ,5.0,Alaska Sourdough


# Get number of reviews per book

In [19]:
numberofReviews = reviews.groupby('BookTitle').count()['Review'].reset_index()
numberofReviews.rename(columns={'Reviews' : 'NumberOfReviews'}, inplace= True)

In [20]:
numberofReviews.head()

,BookTitle,Review
0,""" Film technique, "" and, "" Film acting """,2
1,""" We'll Always Have Paris"": The Definitive Gui...",2
2,"""... And Poetry is Born ..."" Russian Classical...",1
3,"""A Titanic hero"" Thomas Andrews, shipbuilder",8
4,"""A Truthful Impression of the Country"": Britis...",1


# Filter the users and books.

In [ ]:
userId = reviews.groupby('user_id').count()['Review'] > 200 
activeUsers = userId[userId].index